Here we describe how to prepare and make a submission for Python users.  

Disclaimer: the data preprocessing and the model that we will train here are very basic. You should not use it as an example of how to properly train a model (e.g. imputing missing values with a mean or mode is often a bad idea). The sole purpose of this script is to make the submission process more clear.   

In this example we assume that you are using [GitHub Desktop](https://docs.github.com/en/desktop), which is a very convenient tool for updating your github repository. [Here](https://github.com/eyra/fertility-prediction-challenge/wiki) you can find useful links how to use GitHub Desktop for cloning your repository and for update the files in it.  

We also assume that you did all the prerequisite steps described   [here](https://github.com/eyra/fertility-prediction-challenge?tab=readme-ov-file#prerequisites){target="_blank"}.  

Let's imagine that you want to add one predictor to the model that is already in the repository: respondent's gender (variable name "gender_bg", as you found using the codebooks). To produce this model you should use the template functions that are already in the repository:  "clean_df" for preprocessing the data from the "submission.py", "train_save_model" from "training.py" and "predict_outcomes" from "submission.py".   

Overall steps: preprocessing the data —> training and saving the model —> testing on the fake data —> editing "submission.py" and "training.py" accordingly —> submitting.   

1. Create a new script in the environment for python that you normally use, save the file in the folder with your cloned repository. Import necessary packages and read training data and outcome. !IMPORTANT: do not add the data to the folder on your computer with the files from your repository. If you put the data in this folder and then upload all the changes online, the data will be made public in your online github repository. 

```{{python}}
import pandas as pd
from sklearn.linear_model import LogisticRegression
import joblib

# loading features
train = pd.read_csv("path to the data which is NOT your cloned repository\\PreFer_train_data.csv", low_memory = False) # explain low_memory = False
# loading the outcome
outcome = pd.read_csv("path to the data which is NOT your cloned repository\\PreFer_train_outcome.csv", low_memory = False) 
```

2. Copy paste the `clean_df` function from the "submission.py" into this script. Edit the `clean_df` function - add the new variable: 

```{{python}}
def clean_df(df, background_df=None):
    """
    Preprocess the input dataframe to feed the model.
    # If no cleaning is done (e.g. if all the cleaning is done in a pipeline) leave only the "return df" command

    Parameters:
    df (pd.DataFrame): The input dataframe containing the raw data (e.g., from PreFer_train_data.csv or PreFer_fake_data.csv).
    background (pd.DataFrame): Optional input dataframe containing background data (e.g., from PreFer_train_background_data.csv or PreFer_fake_background_data.csv).

    Returns:
    pd.DataFrame: The cleaned dataframe with only the necessary columns and processed variables.
    """

    ## This script contains a bare minimum working example
    # Create new variable with age
    df["age"] = 2024 - df["birthyear_bg"]

    # Imputing missing values in age with the mean
    df["age"] = df["age"].fillna(df["age"].mean())

    # Selecting variables for modelling
    keepcols = [
        "nomem_encr",  # ID variable required for predictions,
        "age"         # newly created variable
        ,"gender_bg"  # <--------ADDED VARIABLE
    ] 

    # Keeping data with variables selected
    df = df[keepcols]

    return df
```

3. Copy paste the `train_save_model` function from the "training.py" and edit it - add the new variable:

```{{python}}
def train_save_model(cleaned_df, outcome_df):
    """
    Trains a model using the cleaned dataframe and saves the model to a file.

    Parameters:
    cleaned_df (pd.DataFrame): The cleaned data from clean_df function to be used for training the model.
    outcome_df (pd.DataFrame): The data with the outcome variable (e.g., from PreFer_train_outcome.csv or PreFer_fake_outcome.csv).
    """
    
    ## This script contains a bare minimum working example
    #random.seed(1) # not useful here because logistic regression deterministic
    
    # Combine cleaned_df and outcome_df
    model_df = pd.merge(cleaned_df, outcome_df, on="nomem_encr")

    # Filter cases for whom the outcome is not available
    model_df = model_df[~model_df['new_child'].isna()]  
    
    # Logistic regression model
    model = LogisticRegression()

    # Fit the model
    model.fit(model_df[['age', 'gender_bg']], model_df['new_child']) # <------- ADDED VARIABLE

    # Save the model
    joblib.dump(model, "model.joblib")

```

4. Preprocess the data using your updated `clean_df` function, and then train the model. Now the model is saved in the same folder. 

In [ ]:
# preprocessing the data
train_cleaned = clean_df(train)

# training and saving the model
train_save_model(train_cleaned, outcome)

5. Test the preprocessing function and model on fake data to see if they will run on the holdout set. If your method does not run on the "fake data", it will not run on the holdout data. [If you "push" your method to Github this test will also be automatically run].  

To do this test you can copy paste the function `predict_outcomes` from the "submission.py". Load the fake data (it is already in your local repository folder) and apply the `predict_outcomes`.

```{{python}}
# load the data
fake = pd.read_csv("PreFer_fake_data.csv") 

def predict_outcomes(df, background_df=None, model_path="model.joblib"):
    """Generate predictions using the saved model and the input dataframe.

    The predict_outcomes function accepts a Pandas DataFrame as an argument
    and returns a new DataFrame with two columns: nomem_encr and
    prediction. The nomem_encr column in the new DataFrame replicates the
    corresponding column from the input DataFrame. The prediction
    column contains predictions for each corresponding nomem_encr. Each
    prediction is represented as a binary value: '0' indicates that the
    individual did not have a child during 2021-2023, while '1' implies that
    they did.

    Parameters:
    df (pd.DataFrame): The input dataframe for which predictions are to be made.
    background_df (pd.DataFrame): The background dataframe for which predictions are to be made.
    model_path (str): The path to the saved model file (which is the output of training.py).

    Returns:
    pd.DataFrame: A dataframe containing the identifiers and their corresponding predictions.
    """

    ## This script contains a bare minimum working example
    if "nomem_encr" not in df.columns:
        print("The identifier variable 'nomem_encr' should be in the dataset")

    # Load the model
    model = joblib.load(model_path)

    # Preprocess the fake / holdout data
    df = clean_df(df, background_df)

    # Exclude the variable nomem_encr if this variable is NOT in your model
    vars_without_id = df.columns[df.columns != 'nomem_encr']

    # Generate predictions from model, should be 0 (no child) or 1 (had child)
    predictions = model.predict(df[vars_without_id])

    # Output file should be DataFrame with two columns, nomem_encr and predictions
    df_predict = pd.DataFrame(
        {"nomem_encr": df["nomem_encr"], "prediction": predictions}
    )

    # Return only dataset with predictions and identifier
    return df_predict

# apply the function to the fake data
predict_outcomes(fake)
```

If you see the predictions, not errors, everything worked.
![](/images/fake_data_predictions.png)
You can now prepare the files for submission, that will be applied to the holdout set.  

6. "submission.py": edit this script by copypasting your clean_df function there. This code will be applied to the holdout data. You don't need to adapt the `predict_outcomes` function in submission.py because the outputs of your model are predicted classes already (i.e., 0s and 1s), and not, for example, probabilities.   

7. prediction model: make sure that your model is saved in the same folder as submission.py under the name model.joblib  

8. "environment.yml": you don't have to edit this file now, because you used only the packages that are already there.  

9. "training.py": copy paste your updated `train_save_model` function and the packages needed for this function to run to this script.  


Now you need to update your online github repository:

10. Go to GitHub Desktop and save changes (i.e. commit).

![](/images/commit.png)


11. Push the changes ("Push origin") (i.e. update your online repository) - press "Push origin" on the upper right.  

12. Now go to the Action page in you online github repository. After a few minutes you'll see if your submission passed the automatic checks.  

13. Submit your method as explained [here](https://github.com/eyra/fertility-prediction-challenge/tree/master#submit-your-method){target="_blank"}. 
